In [2]:
from matplotlib import pyplot as plt
from glob import glob
import numpy as np
import pandas as pd

In [4]:
import json

In [43]:
def load_preds(json_path):
    f = open(json_path)
    preds = json.load(f)
    bbox_collected = []
    score_collected = []
    image_name = preds.keys()
    for k1 in preds.keys():
        temp_box = []
        temp_score = []
        for k2 in preds[k1]:
            box = preds[k1][k2]['bbox']
            x1 = float(box[0])
            y1 = float(box[1])
            x2 = float(box[2])
            y2 = float(box[3])
            temp_box.append([x1, y1, x2, y2])
            temp_score.append(preds[k1][k2]['score'])
        bbox_collected.append(temp_box)  
        score_collected.append(temp_score) 
    return image_name, bbox_collected,score_collected

In [44]:
image_name,pred_bb,_ = load_preds('/p/project/hai_hmgu/workspace/harsha/object_detection/data/predictions/2024-01-30_14-57-02/model_predictions.json')

In [42]:
test_df = pd.read_csv('/p/project/hai_hmgu/workspace/harsha/object_detection/data/orgaquant/test_labels.csv',sep = ',')

In [49]:
base_path = '/p/project/hai_hmgu/workspace/harsha/object_detection/data/orgaquant/test'
gt_bb = []
for img in image_name:
    full_image_path = f'{base_path}/{img}'
    tempdf = test_df[test_df.path == full_image_path]
    boxes = [tempdf.iloc[i,:4].tolist() for i in range(len(tempdf))]
    gt_bb.append(boxes)

In [50]:
gt_bb[0]

[[150, 396, 250, 450], [311, 391, 365, 449], [290, 377, 316, 407]]

In [51]:
def get_iou(a, b, epsilon=1e-5, intersection_check=False):
    x1 = max(a[0], b[0])
    y1 = max(a[1], b[1])
    x2 = min(a[2], b[2])
    y2 = min(a[3], b[3])

    width =  (x2 - x1)
    height = (y2 - y1)

    if (width < 0) or (height < 0):
        if intersection_check:
            return 0.0, False
        else:
            return 0.0
    area_overlap = width * height

    area_a = (a[2] - a[0]) * (a[3] - a[1])
    area_b = (b[2] - b[0]) * (b[3] - b[1])
    area_combined = area_a + area_b - area_overlap

    iou = area_overlap / (area_combined + epsilon)
    if intersection_check:
        return iou, bool(area_overlap)
    else:
        return iou


def calc_conditions(gt_boxes, pred_boxes, iou_thresh=0.5):
    """Function to compute evaluation metrics (TP,FP,FN) based on IOU score.

    Args:
        gt_boxes (List): List of GT bounding boxes.
        pred_boxes (List): List of model predicted bounding boxes.
        iou_thresh (float, optional): Threshold to decide if detection is TP or FP. Defaults to 0.5.

    Returns:
        tp,fp,fn,tpflag (int,int,int,array): Count of number of TPs, FPs, FNs and the flag position of TPs.
    """
    gt_class_ids_ = np.zeros(len(gt_boxes))
    pred_class_ids_ = np.zeros(len(pred_boxes))

    tp, fp, fn = 0, 0, 0
    TPFlag = np.zeros((len(gt_class_ids_),len(pred_class_ids_)))
    for i in range(len(gt_class_ids_)):
        iou = []
        
        for j in range(len(pred_class_ids_)):
            now_iou = get_iou(gt_boxes[i], pred_boxes[j])
            if now_iou >= iou_thresh:
                iou.append(now_iou)
                gt_class_ids_[i] = 1
                pred_class_ids_[j] = 1
            else:
                iou.append(0)

        
        if any(iou):
            tp += 1
            ind = iou.index(max(iou))
            TPFlag[i,ind] = 1
            fp += len(iou) - 1 - iou.count(0)

        
    fn += np.count_nonzero(np.array(gt_class_ids_) == 0)
    fp += np.count_nonzero(np.array(pred_class_ids_) == 0)
    
    return tp, fp, fn, TPFlag
    

def evalMetrics(GTBB,BBColl,iou_thresh):
    """Function to compute and output the evaluated metrics.

    Args:
        GTBB (List): List of GT bounding boxes.
        BBColl (_type_): List of predicted bounding boxes.
        iou_thresh (_type_): IOU threshold.

    Returns:
        TP,FP,FN,TPFLAG (List,List,List,List): TPs,FPs,FNs of all the images stored in a list.
    """
    TP = []
    FP = []
    FN = []
    TPFLAG = []
    for i in range(len(GTBB)):
        if len(GTBB[i]) == 0:
            TP.append(0)
            FP.append(len(BBColl[i]))
            FN.append(0)
            TPFLAG.append([])
        else:
            tp,fp,fn,TPFlag = calc_conditions(GTBB[i],BBColl[i],iou_thresh)
            TP.append(tp)
            FP.append(fp)
            FN.append(fn)
            TPFLAG.append(TPFlag)
            
    return TP,FP,FN,TPFLAG

In [53]:
TP,FP,FN,TPFLAG = evalMetrics(gt_bb,pred_bb,0.5)
TTP = np.sum(TP)
TFP = np.sum(FP)
TFN = np.sum(FN)
PREC = TTP / (TTP + TFP)
RECAL = TTP / (TTP + TFN)
print(f' Total TPs = {TTP}, Total FPs = {TFP}, Total FNs = {TFN}, Overall Precision = {PREC}, Overall Recall = {RECAL} ')


 Total TPs = 1087, Total FPs = 1204, Total FNs = 48, Overall Precision = 0.4744652989960716, Overall Recall = 0.9577092511013215 
